# Host and infer

The preceding training job, when complete, will produce a model export into the configured `S3 output path`.  We will now use that output to create a model which can then be hosted by SageMaker.  The creation of a model can be done programmatically using the code below, alternatively you can create a model manually via the SageMaker web console.

First we will need to create a 'Model' in SageMaker which is a logical construct that associates the export in S3 with a specific algorithm container.  Once the model has been created you will then create an Endpoint Configuration which tells SageMaker the resources needed to run the model in production.  Finally you will deploy an Endpoint using the Endpoint Configuration to create a hosted, secure, RESTful ML service.

## Create a model from training output

### From the console:
From the SageMaker web console `Inference` -> `Model`.  Then click `Create model`.

Give your model a name such as 'xgboost-forecast-model' and then set the `Location of inference code image` to a value such as `644912444149.dkr.ecr.eu-west-2.amazonaws.com/xgboost:latest`.  

Next set the `Location of model artifacts` to a value such as `s3://jasbarto-forecast-lab/output/xgboost-forecast-stock-01-copy-2-copy-12-17-copy-12-17/output/model.tar.gz`

Click `Create model`.

You can always reference the training job details to obtain values for the fields above such as S3 model export location.

### Using the API:

In [ ]:
YOUR_JOB_NAME = ' < TRAINING JOB NAME FROM LAB 04 > '

In [ ]:
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker import get_execution_role
from time import sleep, strftime, gmtime
import pandas as pd

In [ ]:
%%time

role = get_execution_role()

container = get_image_uri(boto3.Session().region_name, 'xgboost')

client = boto3.client ('sagemaker')

job_name = YOUR_JOB_NAME
model_name=job_name + '-model'
print(model_name)

info = client.describe_training_job(TrainingJobName=job_name)
model_data = info['ModelArtifacts']['S3ModelArtifacts']
print(model_data)

primary_container = {
    'Image': container,
    'ModelDataUrl': model_data
}

create_model_response = client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = primary_container)

print(create_model_response['ModelArn'])

### Create endpoint configuration

SageMaker hosts your model as an endpoint.  This endpoint must be configured with which models you would like it to host.  To create an endpoint configuration you can use the code below to create one programmatically or create one manually via the web console.

From the SageMaker web console click `Endpoint configurations` under `Inference`.  Click `Create endpoint configuration`.

Give your endpoint configuration a name such as 'xgboost-model-endpoint-config' and under 'Production variants' click `Add model`.  Select the model you created in the previous step and click `Create endpoint configuration`.

In [ ]:
endpoint_config_name = 'DEMO-XGBoostEndpointConfig-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_config_name)
create_endpoint_config_response = client.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType':'ml.m4.4xlarge',
        'InitialVariantWeight':1,
        'InitialInstanceCount':1,
        'ModelName':model_name,
        'VariantName':'AllTraffic'}])

print("Endpoint Config Arn: " + create_endpoint_config_response['EndpointConfigArn'])

### Deploy endpoint

Finally deploy the configured endpoint so that it can be invoked as a secure, RESTful endpoint.  To deploy the endpoint you can use the code below or create a deployment manually via the web console.

From the SageMaker web console click `Endpoints` under `Inference`.  Click `Create endpoint`.  Give your endpoint a name such as `xgboost-endpoint` and select the endpoint configuration created in the previous step.  Click `Deploy endpoint`.

In [ ]:
%%time

endpoint_name = 'DEMO-XGBoostEndpoint-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_name)
create_endpoint_response = client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name)
print(create_endpoint_response['EndpointArn'])

resp = client.describe_endpoint(EndpointName=endpoint_name)
status = resp['EndpointStatus']
print("Status: " + status)

while status=='Creating':
    sleep(60)
    resp = client.describe_endpoint(EndpointName=endpoint_name)
    status = resp['EndpointStatus']
    print("Status: " + status)

print("Arn: " + resp['EndpointArn'])
print("Status: " + status)

### Invoke the endpoint

Once the SageMaker endpoint has been deployed your trained model is running and ready to perform inference.  Let's test it now using the Boto3 client.  We will use some of the training data produced earlier.  Alternatively you could create a yet unseen data set from a recent month and transform it using the logic in lab 4.  Once transformed you can send this data to the endpoint.

In [ ]:
csv_data_file = !ls data/train/train | head -n1
csv_data_file = "data/train/train/{}".format (csv_data_file[0])
print ("Will invoke endpoint using data from {}".format (csv_data_file))

In [ ]:
!head -n1 {csv_data_file} > /tmp/test_data_single.csv

In [ ]:
%%time
import json
from itertools import islice
import math
import struct
import boto3

sm_client = boto3.client ('runtime.sagemaker')

def predict (features):
    response = sm_client.invoke_endpoint(EndpointName=endpoint_name, 
                                   ContentType='text/csv', 
                                   Body=features)
    result = response['Body'].read()
    return result

file_name = '/tmp/test_data_single.csv' #customize to your test file

with open(file_name, 'r') as f:
    payload = f.read().strip()
label, features = payload.split (',', 1)
print ("Label: {}\nPrediction: {}".format (label, predict (features)))

Using the convenience function defined above iterate over the contents of the `test_data.csv` file and record the prediction for each observation.  The first number of every observation is the label for that observation, store the label, along with the prediction to then compare them as a measure of the model's performance.

In [ ]:
%%time
test_labels = []
test_predictions = []

file_name = '/tmp/test_data_multiple.csv'
!head -n500 {csv_data_file} > {file_name}

with open (file_name, 'r') as f:
    for line in f:
        payload = line.strip ()
        label, features = payload.split (',', 1)
        pred = predict (features)
        test_labels.append (float(label))
        test_predictions.append (float(pred))


Import the `mean_squared_error` function from the `sklearn.metrics` module.  Use the `mean_squared_error` function to calculate the accuracy of the model against the labels and predictions collected in the previous step.


In [ ]:
data = pd.DataFrame ()
data['True_Y'] = test_labels
data['Pred_Y'] = test_predictions

In [ ]:
%matplotlib inline
data.plot.line(figsize=(25,5))

In [ ]:
from sklearn.metrics import mean_squared_error
print ("RMSE: {}".format (mean_squared_error(test_labels, test_predictions)))

In [ ]:
print (test_labels[:10])
print (test_predictions[:10])

---
## Complete

That completes this lab.  You should have trained and created a hosted SageMaker model using the XGBoost algorithm for predicting stock movements using time series analysis.  Now think about how you could improve the accuracy of the model.  What are some of the things you could do to perhaps the data, the algorithm, or the algorithm parameters, to further enhance the performance of your trained model?

### Note: Remove created resources
As part of this lab you will have created an S3 bucket, a SageMaker notebook server, and at least one training job, model, and endpoint.  Please be sure to create each of these to avoid incurring any further charges to your AWS account.